### Notes for Modification

- Wake - Hands overlapping with face.
- Goose - Multiple variations of signs to represent the same thing.
- Nap - Hands overlapping with face/ Eyes close
- Give - Hands are closed.
- After - Hands overlapping
- Mouth - Hand closed, hand over mouth.

Commons Things:
- Closed hands
- Hands over face
- Overlapping landmarks

Solutions:
- Include eye landmarks.
- Backfill missing data with previous landmark data.
- Preserve relative distances between landmarks.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sn

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, GroupShuffleSplit 

from layers.PreprocessLayer import PreprocessLayer
from utils.Utils import print_shape_dtype, pd_read_s3_parquet, upload_file 

import glob
import sys
import os
import math
import gc
import sys
import sklearn
import scipy
import boto3
import io
import wandb
import json

2023-04-14 20:02:11.800819: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 20:02:13.346470: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/tensorflow/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
tf.get_logger().setLevel('INFO')

In [2]:
with open("./config/config.json") as fp:
    config = json.load(fp)

In [3]:
s3_client = boto3.client(
    "s3"
)

In [4]:
AWS_S3_BUCKET = "w251-asl-data"
TRAIN_CSV_FILE = "raw-data/train.csv"

In [5]:
train_file = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=TRAIN_CSV_FILE)

In [6]:
train = pd.read_csv(train_file.get("Body"))

N_SAMPLES = len(train)
print(f'N_SAMPLES: {N_SAMPLES}')

N_SAMPLES: 94477


In [7]:
# Get complete file path to file
def get_file_path(path):
    return f'{AWS_S3_BUCKET}/raw-data/{path}'

train['file_path'] = train['path'].apply(get_file_path)

In [8]:
# Add ordinally Encoded Sign (assign number to each sign name)
train['sign_ord'] = train['sign'].astype('category').cat.codes

# Dictionaries to translate sign <-> ordinal encoded sign
SIGN2ORD = train[['sign', 'sign_ord']].set_index('sign').squeeze().to_dict()
ORD2SIGN = train[['sign_ord', 'sign']].set_index('sign_ord').squeeze().to_dict()

In [ ]:
display(train.head(30))
display(train.info())

In [9]:
# Source: https://www.kaggle.com/competitions/asl-signs/overview/evaluation
ROWS_PER_FRAME = 543  # number of landmarks per frame
#w251-asl-data/raw-data/train_landmark_files/28656/3311214787.parquet

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y']
    data = pd_read_s3_parquet(pq_path[14:], AWS_S3_BUCKET, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [10]:
preprocess_layer = PreprocessLayer(config["INPUT_SIZE"])

2023-04-14 20:02:24.584683: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-14 20:02:24.969414: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-14 20:02:24.971881: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [11]:
"""
    face: 0:468
    left_hand: 468:489
    pose: 489:522
    right_hand: 522:544
        
"""
def get_data(file_path):
    # Load Raw Data
    data = load_relevant_data_subset(file_path)
    # Process Data Using Tensorflow
    data = preprocess_layer(data)
    
    return data

In [12]:
version = config["DATA_VERSION"]

# Get the full dataset
def preprocess_dataset():
    # Create arrays to save data
    X = np.zeros([N_SAMPLES, config["INPUT_SIZE"], config["N_COLS"] * config["N_DIMS"]], dtype=np.float32)
    y = np.zeros([N_SAMPLES], dtype=np.int32)
    NON_EMPTY_FRAME_IDXS = np.full([N_SAMPLES, config["INPUT_SIZE"]], -1, dtype=np.float32)

    for row_idx, (file_path, sign_ord) in enumerate(tqdm(train[['file_path', 'sign_ord']].values)):
        if row_idx % 5000 == 0:
            print(f'Generated {row_idx}/{N_SAMPLES}')

        data, non_empty_frame_idxs = get_data(file_path)
        X[row_idx] = data
        y[row_idx] = sign_ord
        NON_EMPTY_FRAME_IDXS[row_idx] = non_empty_frame_idxs
        if np.isnan(data).sum() > 0:
            print(row_idx)
            return data
    
    # Save X/y
    np.save('X.npy', X)
    np.save('y.npy', y)
    np.save('NON_EMPTY_FRAME_IDXS.npy', NON_EMPTY_FRAME_IDXS)
    
    # Put to S3
    upload_file("./X.npy", AWS_S3_BUCKET, f'processed-data/v{version}/X.npy')
    upload_file("./y.npy", AWS_S3_BUCKET, f'processed-data/v{version}/y.npy')
    upload_file("./NON_EMPTY_FRAME_IDXS.npy", AWS_S3_BUCKET, f'processed-data/v{version}/NON_EMPTY_FRAME_IDXS.npy')
    
    return X, y, NON_EMPTY_FRAME_IDXS

In [13]:
X, y, NON_EMPTY_FRAME_IDXS = preprocess_dataset()

print_shape_dtype([X, y, NON_EMPTY_FRAME_IDXS], ['X', 'y', 'NON_EMPTY_FRAME_IDXS'])
print(f'# NaN Values X: {np.isnan(X).sum()}')

  0%|          | 0/94477 [00:00<?, ?it/s]

Generated 0/94477
Generated 5000/94477
Generated 10000/94477
Generated 15000/94477


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Generated 35000/94477
Generated 40000/94477
Generated 45000/94477
Generated 50000/94477
Generated 55000/94477
Generated 60000/94477
Generated 65000/94477
Generated 70000/94477
Generated 75000/94477
Generated 80000/94477
Generated 85000/94477
Generated 90000/94477
X shape: (94477, 38, 454), dtype: float32
y shape: (94477,), dtype: int32
NON_EMPTY_FRAME_IDXS shape: (94477, 38), dtype: float32
# NaN Values X: 0


In [ ]:
display(pd.Series(y).value_counts().to_frame('Class Count').iloc[[0,1,2,3,4, -5,-4,-3,-2,-1]])

In [ ]:
X = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=f'processed-data/v{version}/X.npy')
X = np.load(io.BytesIO(X['Body'].read()))

y = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=f'processed-data/v{version}/y.npy')
y = np.load(io.BytesIO(y['Body'].read()))

NON_EMPTY_FRAME_IDXS = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=f'processed-data/v{version}/NON_EMPTY_FRAME_IDXS.npy')
NON_EMPTY_FRAME_IDXS = np.load(io.BytesIO(NON_EMPTY_FRAME_IDXS['Body'].read()))

In [ ]:
data = pd_read_s3_parquet("raw-data/train_landmark_files/28656/1000106739.parquet", AWS_S3_BUCKET)

In [ ]:
data[(data['frame']==29) & (data["type"] == "pose")]

In [ ]:
data[(data['frame']==29) & (data.index.isin(POSE_IDXS0))].plot.scatter(x='x',y='y', marker='.')